In [1]:
import re
import shutil
from pathlib import Path
import glob
import json
import os
from tqdm import tqdm

In [2]:
def read_line(line):
    image_id, template_path, pred = re.search("image name:(.*).*template_path: (.*).*target:.*predicted:.*(0|1)", line).groups()
    return image_id.strip(), template_path.strip(), pred.strip()

def check_sum(my_dict):
    k = 0
    for i in my_dict:
        k += len(my_dict[i].keys())
    return k
                 

In [9]:
read_line("image name:6369e182-af80-4cbe-85d2-7bd4547a1653	template_path: 15364270390863969pic.jpg target: 1 predicted: 1")

('6369e182-af80-4cbe-85d2-7bd4547a1653', '15364270390863969pic.jpg', '1')

In [3]:
##rule
rule_results = {}
with open("/data/home/yanghanlong/all_results_rule.txt", 'r') as f:
    rule_data = f.readlines()
    for d in rule_data:
        image_id, template_path, pred = read_line(d)
        if image_id not in rule_results.keys():
            rule_results[image_id] = {}
        rule_results[image_id][template_path] = pred

In [4]:
check_sum(rule_results)

5553

In [5]:
##model
model_results = {}
with open("/data/home/yanghanlong/all_results_model.txt", 'r') as f:
    model_data = f.readlines()
    for d in model_data:
        image_id, template_path, pred = read_line(d)
        if image_id not in model_results.keys():
            model_results[image_id] = {}
        model_results[image_id][template_path] = pred

In [6]:
##diff
diff = []
for i in model_results:
    for j in model_results[i]:
        if model_results[i][j] == '1' and rule_results[i][j] == '1':
            # print(i,j)
            diff.append((i,j))

In [33]:
4899/(524+4899)

0.9033745159505808

In [7]:
##extract blended.jpg into one folder according to image id and template path
def extract_jpg(image_id,tem_path, save_dir):
    ##get template index
    with open(f"/data/home/yanghanlong/data/image_retrieval_for_eval_ia/anno/{image_id}.json", 'r') as f:
        candidates = json.load(f)["candidate_template_images"]
    tem_index = candidates.index(tem_path)
    ##find blended.jpg
    blended = glob.glob(f"/data/home/yanghanlong/results/image_align/image_retrieval_for_eval_ia/{image_id}/page_{tem_index}*/blended.*")
    matches = glob.glob(f"/data/home/yanghanlong/results/image_align/image_retrieval_for_eval_ia/{image_id}/page_{tem_index}*/matches.*")
    assert len(blended) != 0
    blended = blended[0]; matches = matches[0]
    ##copy to save_dir
    tem_name = Path(tem_path).stem
    save_path = Path(save_dir).joinpath(f"{image_id}_{tem_name}")
    os.makedirs(save_path, exist_ok=True)
    shutil.copy(blended, save_path)
    shutil.copy(matches, save_path)


In [8]:
for i,j in tqdm(diff):
    extract_jpg(i, j, "/data/home/yanghanlong/results/compare_rule_with_model/all_received")

100%|██████████| 4899/4899 [00:14<00:00, 331.96it/s]
